# Vacinometro Estado SP

In [9]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime
import numpy as np
import pathlib
import shutil

def ontem(hoje):
    distD = int(hoje.strftime('%d'))-1
    distM = int(hoje.strftime('%m'))
    distA = 21
    if distD < 10:
        return '0{}/0{}/{}'.format(str(distD),str(distM),str(distA))
    return '{}/0{}/{}'.format(str(distD),str(distM),str(distA))

def dia_de_anteontem(hoje):
    dist = int(hoje.strftime('%d'))-2
    if dist < 10:
        return '0'+str(dist)
    else:
        return str(dist)

def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return '0'+str(dist)
    else:
        return str(dist)


def download_wait(directory, timeout, nfiles=None):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

def download_check(directory, file_name):
    seconds = 0
    no_dl = True
    files = os.listdir(directory)
    for fname in files:
        if fname.endswith('{}'.format(file_name)):
            return True
    raise ValueError("Nao está")

lista=['isolamento','vacinometro']
home = Path.home()
parent = pathlib.Path().absolute()
d = Path(home,'Downloads')
cidade='piracicaba'
t=Path(parent,"bases/{}/".format(cidade))
source_dir = d
target_dir = t
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
path_download19 = Path(parent,"bases/{}/vacinas.csv".format(cidade))
path_download20 = Path(parent,"bases/{}/isolamento.csv".format(cidade))

hoje = datetime.now()
dhojes = list(range(1,28))
mes= '04'
hora = hoje.strftime('%H')
for dhoje in dhojes:
    if dhoje <10:
        dhoje='0'+str(dhoje)
    else:
        dhoje=str(dhoje)
    noite=False
    for item in lista:
        driver = webdriver.Chrome(path_win)


        link1 = "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_{}.csv".format(mes,mes,dhoje,item)
        link2= "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_{}.csv".format(mes,mes,dontem,item)


        t1=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,dhoje,item))
        t2=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,dontem,item))
        t3=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,danteontem,item))

        d1=Path(d,"2021{}{}_{}.csv".format(mes,dhoje,item))
        d11=Path(d,"2021{}{}_{} (1).csv".format(mes,dhoje,item))
        d2=Path(d,"2021{}{}_{}.csv".format(mes,dontem,item))
        d21=Path(d,"2021{}{}_{} (1).csv".format(mes,dontem,item))
        d3=Path(d,"2021{}{}_{}.csv".format(mes,danteontem,item))
        d31=Path(d,"2021{}{}_{} (1).csv".format(mes,danteontem,item))


        arq1 = "2021{}{}_{}.csv".format(mes,dhoje,item)
        arq2 = "2021{}{}_{}.csv".format(mes,dontem,item)
        arq3 = "2021{}{}_{}.csv".format(mes,danteontem,item)

        try:
            noite=True
            data_str = '2021-{}-{}'.format(mes,dhoje)
            data=  '2021-{}-{}'.format(mes,dhoje)
            if (item == 'isolamento'):
                driver.get(link1)
                driver.implicitly_wait(10)
                download_wait(d,130)
                download_check(d,'isolamento.csv')
                driver.quit()
            elif (item == 'vacinometro'):
                driver.get(link1)
                driver.implicitly_wait(10)
                download_wait(d,130)
                download_check(d,'vacinometro.csv')
                driver.quit()
            elif (item == 'leitos_ocupados_por_unidade_hospitalar'):
                driver.get(link1)
                driver.implicitly_wait(10)
                download_wait(d,130)
                download_check(d,'leitos_ocupados_por_unidade_hospitalar.csv')
                driver.quit()
            else:
                raise ValueError("Nao está")

        except:
            noite=False
            data = ontem(hoje)
            data= datetime.strptime(data, '%d/%m/%y')
            data = '{}-0{}-{}'.format(data.year, data.month, data.day)
            if (item == 'isolamento'):
                driver.get(link2)
                driver.implicitly_wait(10)
                download_wait(d,130)
                driver.quit()
            else:
                driver.get(link2)
                driver.implicitly_wait(10)
                download_wait(d,30)
                driver.quit()


        file_names = os.listdir(source_dir)

        for file_name in file_names:
            if noite:
                if (file_name==arq1) and not(t1.is_file()):
                    shutil.move(os.path.join(source_dir, file_name), target_dir)
            else:
                if file_name==arq2 and not(t2.is_file()):
                    shutil.move(os.path.join(source_dir, file_name), target_dir)
                continue

#         if (t2.is_file())&(t1.is_file())& noite :
#             os.remove(t2)

#         elif (t3.is_file()) & (t2.is_file()):
#             os.remove(t3)

#         if d1.is_file():
#             os.remove(d1)
#         if d2.is_file():
#             os.remove(d2)
#         if d3.is_file():
#             os.remove(d3)
#         if d11.is_file():
#             os.remove(d11)
#         if d21.is_file():
#             os.remove(d21)
#         if d31.is_file():
#             os.remove(d31)

        if noite:
            try:
                df = pd.read_csv(t1,';')
            except FileNotFoundError:
                df = pd.read_csv(t2,';')
        else:
            df = pd.read_csv(t2,',')

        if item=='isolamento':
            vacinas =pd.DataFrame()
            try:
                df = df[df['Município1'] == 'PIRACICABA']
            except:
                df = df[df.iloc[:,0] == 'PIRACICABA']

            vacinas = vacinas.append(df,ignore_index=True)
            vacinas.to_csv(path_download20,index=False)
        elif item=='vacinometro':
            vacinas = pd.read_csv(path_download19)
            try:
                df = df[df['Municipio'] == 'PIRACICABA']
                df.columns=['Municipio','Dose','Contagem de Id Vacinacao']
                df['Data'] = data
            except:
                df = df[df.iloc[:,0] == 'PIRACICABA']
                df.columns=['Municipio','Dose','Contagem de Id Vacinacao']
                df['Data'] = data
            if vacinas['Data'].iloc[-1]!= data:   
                if len(df)<2:
                    if df.Dose.iloc[0]=='1° Dose':
                        linha=df[df.Dose=='1° Dose']
                        linha['Dose']='2° Dose'
                        linha['Contagem de Id Vacinacao']=vacinas['Contagem de Id Vacinacao'].iloc[-2]
                        vacinas = vacinas.append(linha,ignore_index=True)
                        vacinas = vacinas.append(df,ignore_index=True)
                    else:
                        vacinas = vacinas.append(df,ignore_index=True)
                        linha=df[df.Dose=='2° Dose']
                        linha['Dose']='1° Dose'
                        linha['Contagem de Id Vacinacao']=vacinas['Contagem de Id Vacinacao'].iloc[-1]
                        vacinas = vacinas.append(linha,ignore_index=True)
                else:
                    vacinas = vacinas.append(df,ignore_index=True)
            else:
                continue
            vacinas.to_csv(path_download19,index=False)
        else:
            continue

    

<ipython-input-9-fe0ee7b7a7ea>:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data'] = data


# Casos Prefeitura

In [23]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

import pathlib
cidade='piracicaba'
parent = pathlib.Path().absolute()
path_download15 = Path(parent,"bases/{}/covid.csv".format(cidade))

def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return '0'+str(dist)
    else:
        return str(dist)

hoje = datetime.now()
dhoje = hoje.strftime('%d')
dontem = dia_de_ontem(hoje)
mes= hoje.strftime('%m')
hora = hoje.strftime('%H')

covid = pd.read_csv(path_download15)
covid.drop('Unnamed: 0',axis=1,inplace=True)
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)
if int(hora) >=18:
    dias = list(range(22,28))
    meses =['{}'.format(mes)]
else:
    dias = list(range(22,28))
    meses =['{}'.format(mes)]
for m in meses:
    for d in dias:
        if d<=12:
            link = 'http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx/3'
        elif d>12 and d<=19:
            link = 'http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx/2'
        else:
            link = 'http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx'
        if d <10:
            d='0'+str(d)
        else:
            d=str(d)
        print(link)
        driver.get(link)
        div_prim = driver.find_elements_by_class_name('imagenet-topico')
        for item in div_prim:
            if item.text=='› {}/{}/2021 - PLANTÃO CORONAVÍRUS- {}/{}/2021'.format(d,m,d,m):
                item.click()
                break
            elif item.text=='› {}/{}/2021 - PLANTÃO CORONAVÍRUS - {}/{}/2021'.format(d,m,d,m):
                item.click()
                break
            else: 
                continue
        div= driver.find_element_by_id('imagenet-conteudo')
        novos_obitos = 'NA'
        casos_confirmados = 'NA'
        casos_suspeitos = 'NA'
        casos_descartados = 'NA'
        casos_recuperados = 'NA'
        em_tratamento = 'NA'
        obitos = 'NA'
        homens = 'NA'
        mulheres = 'NA'
        for i in range(len(div.text.split('\n'))):
            if (len(div.text.split('\n')[i].split())<5)&(len(div.text.split('\n')[i].split())>1):
                if div.text.split('\n')[i].split()[-1] =='confirmados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_confirmados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_confirmados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='suspeitos':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_suspeitos = div.text.split('\n')[i].split()[0][:4]
                    else:
                        casos_suspeitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='descartados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_descartados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_descartados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='recuperados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_recuperados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_recuperados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='tratamento':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        em_tratamento = div.text.split('\n')[i].split()[0][:4]
                    else:
                        em_tratamento = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbitos':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[0][:3]
                    else:
                        obitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbito':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[0][:3]
                    else:
                        obitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[0] =='Óbitos-':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[-1]
                    else:
                        obitos = div.text.split('\n')[i].split()[-1]
                elif div.text.split('\n')[i].split()[0] =='Óbitos':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[-1]
                    else:
                        obitos = div.text.split('\n')[i].split()[-1]
            elif len(div.text.split('\n')[i].split())>1:
                if div.text.split('\n')[i].split()[1] =='homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher':
                    mulheres = div.text.split('\n')[i].split()[0]
        dicionario ={'Total Casos Confirmados': int(casos_confirmados.replace('.','')),
                     'Total Casos Suspeitos':int(casos_suspeitos.replace('.','')),
                     'Total Casos Descartados':int(casos_descartados.replace('.','')),
                     'Total Casos Recuperados':int(casos_recuperados.replace('.','')),
                     'Total Casos em Tratamento':int(em_tratamento.replace('.','')),
                     'Total Óbitos':int(obitos.replace('.','')),
                     'Homens':int(homens.replace('.','')),
                     'Mulheres':int(mulheres.replace('.','')),
                     'Data':'{}/{}/2021'.format(d,m)}
        if covid['Data'].iloc[-1]!=dicionario['Data']:
            covid = covid.append(dicionario, ignore_index=True)     
            covid['Óbitos Diários'].iloc[-1]=covid['Total Óbitos'].iloc[-1]-covid['Total Óbitos'].iloc[-2]
            covid['Casos Diários'].iloc[-1]=covid['Total Casos Confirmados'].iloc[-1]-covid['Total Casos Confirmados'].iloc[-2]

        covid['Média Móvel Casos'] = covid['Casos Diários'].rolling(7).mean()
        covid['Média Móvel Óbitos'] = covid['Óbitos Diários'].rolling(7).mean()
        covid['Média Móvel Tratamento'] = covid['Total Casos em Tratamento'].rolling(7).mean()
        covid['Média Móvel Suspeitos'] = covid['Total Casos Suspeitos'].rolling(7).mean()
        
covid.to_csv(path_download15)
driver.quit()

http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx
http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx
http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx
http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx
http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx
http://www.piracicaba.sp.gov.br/categoria/principais+noticias.aspx


In [15]:

from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime
import pathlib
import shutil

def download_wait(directory, timeout, nfiles=None):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

def download_check(directory, file_name):
    seconds = 0
    no_dl = True
    files = os.listdir(directory)
    for fname in files:
        if fname.endswith('{}'.format(file_name)):
            return True
    raise ValueError("Nao está")
    
cidade='piracicaba'
home = Path.home()
d = Path(home,'Downloads')
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")





vacinas = pd.DataFrame()
datas=[('15','02'),('16','02'),('17','02'),('18','02'),('18','02'),('19','02'),('20','02'),('21','02'),
       ('22','02'),('23','02'),('24','02'),('25','02'),('26','02'),('27','02'),
       ('01','03'),('02','03'),('03','03'),('04','03'),('05','03'),('06','03')]
for dia,mes in datas:
    data_str='{}/{}/21'.format(dia,mes)
    data=datetime.strptime(data_str, '%d/%m/%y')
    driver = webdriver.Chrome(path_win)
    d1=Path(d,"2021{}{}_vacinometro.csv".format(mes,dia))
    link1 = "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_vacinometro.csv".format(mes,mes,dia)
    driver.get(link1)
    driver.implicitly_wait(10)
    download_wait(d,130)
    
    driver.quit()
    df = pd.read_csv(d1,';')
    try:
        df = df[df['Municipio'] == 'PIRACICABA']
        df['Data'] = data
    except:
        df = df[df.iloc[:,0] == 'PIRACICABA']
        df['Data'] = data
    vacinas = vacinas.append(df,ignore_index=True)
parent = pathlib.Path().absolute()
path_download19 = Path(parent,"bases/{}/vacinas.csv".format(cidade))
vacinas.to_csv(path_download19)

KeyboardInterrupt: 

# Sessões Câmara

In [26]:
# ------------IMPORTS-------------
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime
import pathlib
import camelot
# -----------FUNCTIONS------------

def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return '0'+str(dist)
    else:
        return str(dist)
    
# -------------PATHS-----------------

cidade='piracicaba'
parent = pathlib.Path().absolute()
path_download15 = Path(parent,"bases/{}/sessoes.csv".format(cidade))
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)
base_df = pd.read_csv(path_download15)

# --------------TIME (HOURS,DAYS,MONTHS)-------------------
hoje = datetime.now()
dhoje = hoje.strftime('%d')
dontem = dia_de_ontem(hoje)
mes= hoje.strftime('%m')
hora = hoje.strftime('%H')


# --------------LOGIC-------------------


# -----------set dias and meses------------------
dias = ['25']
meses =['03']

    
# -----------run in selected dias e meses
for m in meses:
    for d in dias:
        data='2021-{}-{}'.format(m,d)
        link = 'https://siave.camarapiracicaba.sp.gov.br/sessoes'  # fixed link
        print(link)
        driver.get(link)
        div_prim = driver.find_elements_by_xpath("//table/tbody/tr")
        achou=False
        endereco=False
        for i in range(len(div_prim)): #run trought the list of news of Prefeitura
            if (len(div_prim[i].text.split()) >= 6) and (div_prim[i].text.split()[-2]=='{}/{}/2021'.format(d,m)):  #select the dia and mes
                j=i+1
                item = driver.find_elements_by_xpath("//table/tbody/tr[{}]/td".format(j))
                link = driver.find_element_by_link_text('{}'.format(item[0].text))
                link.click()
                achou=True
                break
            else: 
                continue
        if achou:
            link = driver.find_element_by_link_text('RELATÓRIO DE VOTAÇÃO')
            link.click()
            pdf= driver.find_elements_by_xpath("//table/tbody/tr[1]/td[2]")
            pdf[0].click()
            url = driver.current_url
            endereco=True
        if achou==False:
            driver.quit()
            break
            
        if endereco:
            file = '{}'.format(url)
            todo = camelot.read_pdf(file,pages='all')
            for i in range(len(todo)):
                print('{}/{}'.format(j,i))
                if todo[i].df.iloc[0,0].split('\n')[6]!='0':
                    vereador = todo[i].df.iloc[0,0].split('\n')[15]
                    if len(todo[i].df.iloc[0,0].split('\n'))<=29:
                        ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]
                    elif len(todo[i].df.iloc[0,0].split('\n'))==30:
                        ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]
                    elif len(todo[i].df.iloc[0,0].split('\n'))==31:
                        ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]
                    elif len(todo[i].df.iloc[0,0].split('\n'))==32:
                        ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]+ todo[i].df.iloc[0,0].split('\n')[21]
                    elif len(todo[i].df.iloc[0,0].split('\n'))==33:
                        ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]+ todo[i].df.iloc[0,0].split('\n')[21]+ todo[i].df.iloc[0,0].split('\n')[22]
                    presentes = int(todo[i].df.iloc[0,1].split('\n')[0])
                    ausentes = int(todo[i].df.iloc[0,1].split('\n')[2])
                    if len(todo[i].df.iloc[1,1].split('\n')[0])<3:
                        sim = int(todo[i].df.iloc[1,1].split('\n')[0])
                        nao = int(todo[i].df.iloc[1,1].split('\n')[2])
                        abst= int(todo[i].df.iloc[1,1].split('\n')[4])
                    else:
                        sim = int(todo[i].df.iloc[1,1].split('\n')[1])
                        nao = int(todo[i].df.iloc[1,1].split('\n')[0].split()[0])
                        abst= int(todo[i].df.iloc[1,1].split('\n')[0].split()[1])
                    status=todo[i].df.iloc[2,1]
                    tempo_vot = todo[i].df.iloc[0,0].split('\n')[-3]
                    dicionario = {'Data':data,'Propositor':vereador,'Ementa':ementa,'Presentes':presentes,
                                'Votos a Favor':sim,'Votos Contrários':nao, 'Status':status,'Tempo de Votação':tempo_vot}
                    base_df=base_df.append(dicionario,ignore_index=True)
                    driver.quit()
                else:
                    continue
base_df.to_csv(path_download15,index=False)                    

https://siave.camarapiracicaba.sp.gov.br/sessoes
7/0
7/1
7/2


In [2]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime
import numpy as np
import pathlib
import shutil
parente = pathlib.Path().absolute()

path_download19 = Path(parente,"bases/piracicaba/vacinas.csv")

vacinas=pd.read_csv(path_download19)


soma_vac = vacinas.groupby('Data').sum()
soma_vac.sort_values('Data',inplace=True)

vac_t=soma_vac['Contagem de Id Vacinacao'].tolist()
data_t=soma_vac.index.tolist()


vacinas2=vacinas.iloc[5:]
prid=vacinas2[vacinas2['Dose']=='1° Dose']
sd=vacinas2[vacinas2['Dose']=='2° Dose']
sd['DosesDiarias']=sd['Contagem de Id Vacinacao']-sd['Contagem de Id Vacinacao'].shift(1)
prid['DosesDiarias']=prid['Contagem de Id Vacinacao']-prid['Contagem de Id Vacinacao'].shift(1)
data_p=prid['Data'].tolist()[2:]
segdose=sd['DosesDiarias'].tolist()[1:]
pridose=prid['DosesDiarias'].tolist()[2:]
numRequerido=[407252*2]*len(data_t)

<ipython-input-2-36dbce1b78be>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sd['DosesDiarias']=sd['Contagem de Id Vacinacao']-sd['Contagem de Id Vacinacao'].shift(1)
<ipython-input-2-36dbce1b78be>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prid['DosesDiarias']=prid['Contagem de Id Vacinacao']-prid['Contagem de Id Vacinacao'].shift(1)
